In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.optimizers import Adam, SGD, RMSprop
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle                      
import cv2                                 
import tensorflow as tf                

In [ ]:
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

# This size needed for EfficientNetB0
IMAGE_SIZE = (224, 224)

def load_data():
    datasets = ['/kaggle/input/face-expression-recognition-dataset/images/train/']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            count = 0
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
#                 image = cv2.imread(img_path)
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
                # Taking 1k images from of each type
                # bcause Notebook getting out of memory error
                count = count + 1
                if count == 1000:
                    break
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

(images, labels) = load_data()[0]

In [ ]:
# # Memory taken by array
print((images.nbytes/1024)/1024)

permutation  = np.random.permutation(len(images))

shuffled_images = images[permutation]
shuffled_labels = labels[permutation]

from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(shuffled_images, shuffled_labels, test_size=0.1)
train_images = train_images / 255.0 
test_images = test_images / 255.0

# # Memory taken by array
# (images.nbytes/1024)/1024

In [ ]:
def display_random_image(class_names, images, labels):   
    index = np.random.randint(images.shape[0])
    plt.figure()
    plt.imshow(images[index])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title('Image #{} : '.format(index) + class_names[labels[index]])
    plt.show()
    
display_random_image(class_names, train_images, train_labels)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense

# base_model = EfficientNetB0(
#     weights='imagenet', 
#     input_shape=(224, 224, 3),
#     include_top=False
#     ) 

# base_model = VGG16(
#     weights='imagenet', 
#     input_shape=(224, 224, 3),
#     include_top=False
#     ) 

# base_model.trainable = False

In [ ]:
# inputs = Input(shape=(224, 224, 3))
# model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

# # Freeze the pretrained weights
# model.trainable = False

# # Rebuild top
# x = GlobalAveragePooling2D(name="avg_pool")(model.output)
# x = BatchNormalization()(x)

# x = Dropout(0.2)(x)
# outputs = Dense(7, activation="softmax")(x)

# # Create the final model
# model = Model(inputs, outputs)

# model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
# model.summary()

inputs = Input(shape=(224, 224, 3))
base_model = VGG16(
    weights='imagenet', 
    input_tensor=inputs,
    include_top=False
    ) 

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = BatchNormalization()(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(50, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(7, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary(show_trainable=True)

In [ ]:
history = model.fit(train_images, train_labels, epochs=10, validation_split = 0.2)

In [ ]:
plt.plot(history.history['accuracy'], label = "acc")
plt.plot(history.history['val_accuracy'], label = "val_acc")
plt.title("train_acc vs val_acc")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend()

In [ ]:
base_model.trainable = True
model.summary(show_trainable=True)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

epochs = 1
print("Fitting the end-to-end model")
history = model.fit(train_images, train_labels, epochs=1, validation_split = 0.2)

In [ ]:
predictions = model.predict(test_images)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability

# Display Random Five predicted labels
for i in range(5):
    display_random_image(class_names, test_images, pred_labels) 

In [ ]:
# def unfreeze_model(model):
#     # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
#     for layer in model.layers[-20:]:
#         if not isinstance(layer, BatchNormalization):
#             layer.trainable = True

#     optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
#     model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])


# unfreeze_model(model)
# print(model.summary())

<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:skyblue;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h2 style='padding: 20px;
              color:black;
              text-align:center;'>
        IF YOU LIKED THE NOTEBOOK PLEASE DO UPVOTE!  
    </h2>
    </div>